In [ ]:
import pandas as pd
import re
import numpy as np
from collections import Counter

# Load the Mahindra CSV file
mahindra_data = pd.read_csv('/home/bipin/Documents/kotak/car_model/manufacturer_data/MAHINDRA.csv')
mahindra_data.head()

# Function to clean and normalize model names
def clean_model(text):
    if pd.isna(text) or text is None:
        return ""
    # Convert to string
    text = str(text).upper()
    # Remove common prefixes and company names
    text = re.sub(r'MAHINDRA\s+[&\+]\s+MAHINDRA\s+(LIMITED|LTD)?,?\s*', '', text)
    text = re.sub(r'MAHINDRA\s+', '', text)
    text = re.sub(r'MAH\s+[&\+]\s+MAH\s*', '', text)
    text = re.sub(r'MAH[\.\s]', '', text)
    text = re.sub(r'M\s*[&\+]\s*M\s+', '', text)
    # Remove special characters and extra spaces
    text = re.sub(r'[^\w\s-]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Create a list of cleaned model names
mahindra_data['clean_model'] = mahindra_data['rc_maker_model'].apply(clean_model)

# Filter out too generic or empty model names
generic_terms = ['LTD', 'BS', 'IV', 'III', 'BS-IV', 'BS-III', 'BSIV', 'BSIII', 'STD', 'LIMITED']
filtered_models = mahindra_data['clean_model'].apply(
    lambda x: x if x and all(term not in x.split() for term in generic_terms) else np.nan
)

# Count model name frequencies
model_counts = Counter([m for models in filtered_models.dropna() for m in models.split()])
model_counts = {k: v for k, v in model_counts.items() if v >= 10}  # Keep only frequent terms

# Print top model keywords by frequency
print("Top model keywords by frequency:")
for model, count in sorted(model_counts.items(), key=lambda x: x[1], reverse=True)[:30]:
    print(f"{model}: {count}")

# Define primary Mahindra model keywords
mahindra_model_keywords = [
    # SUVs
    "SCORPIO", "XUV500", "XUV300", "THAR", "BOLERO", "TUV300", "KUV100", 
    "MARAZZO", "XYLO", "XUV700", "QUANTO", "ALTURAS", "VERITO", 
    
    # Trucks and Commercial
    "MAXIMO", "MAXXIMO", "SUPRO", "JEETO", "ALFA", "LOADKING", "BMT", "BLAZO", 
    "FURIO", "TREO", "IMPERIO", "GENIO", "BOLERO PICKUP", "BOLERO MAXI", "JAYO",
    
    # Legacy/Discontinued Models
    "LOGAN", "CL500", "SAVARI", "INVADER", "MARSHAL", "ARMADA", "CJ", 
    
    # Variants/Trims
    "SLE", "SLX", "LX", "DX", "VLX", "ZLX", "PLUS", "CAMPER", "FESTARA", "MAXX",
    
    # Engine/Drivetrain Identifiers  
    "MHAWK", "CRDE", "DICR", "MDI", "2WD", "4WD", "DI", "TC",
    
    # Common model suffixes/variants
    "NEO", "VIBE", "PIK-UP", "PICK-UP", "PIKUP", "PICKUP", "PIKLUP", "TOUR"
]

# Create alias map for common misspellings and variations
mahindra_alias_map = {
    # Scorpio variations
    "SCORIPO": "SCORPIO",
    "SCORIPO": "SCORPIO",
    "SCORPEO": "SCORPIO",
    "SCORPION": "SCORPIO",
    "SCORPI": "SCORPIO",
    "SCORP": "SCORPIO",
    "SCOR": "SCORPIO",
    "SC RF": "SCORPIO",
    "SC EX": "SCORPIO",
    "SCO EX": "SCORPIO",
    "SCEX": "SCORPIO",
    
    # Bolero variations
    "BALERO": "BOLERO",
    "BELERO": "BOLERO",
    "BOLARO": "BOLERO",
    "BOL": "BOLERO",
    "BOLORO": "BOLERO",
    
    # XUV variations
    "XUV 500": "XUV500",
    "XUV5OO": "XUV500",
    "XUV-500": "XUV500",
    "XUV 300": "XUV300",
    "XUV3OO": "XUV300",
    "XUV-300": "XUV300",
    "XUV 700": "XUV700",
    "XUV7OO": "XUV700",
    "XUV-700": "XUV700",
    
    # TUV variations
    "TUV 300": "TUV300",
    "TUV-300": "TUV300",
    
    # KUV variations
    "KUV 100": "KUV100",
    "KUV-100": "KUV100",
    "KU100": "KUV100",
    
    # Xylo variations
    "XYLLO": "XYLO",
    "XYLO": "XYLO",
    "XILO": "XYLO",
    
    # Maxximo variations
    "MAXXMO": "MAXXIMO",
    "MAXIMO": "MAXXIMO",
    "MAXIMMO": "MAXXIMO",
    
    # Pickup variations
    "PICK UP": "PICKUP",
    "PIK UP": "PICKUP",
    "PIC UP": "PICKUP",
    "PIKUP": "PICKUP",
    "PIKLUP": "PICKUP",
    
    # Commercial vehicles
    "LOADKING": "LOADKING",
    "LOAD KING": "LOADKING",
    "LOADKNG": "LOADKING",
    
    # Model + Trim combinations
    "BOLERO SLX": "BOLERO",
    "BOLERO LX": "BOLERO",
    "BOLERO ZLX": "BOLERO",
    "SCORPIO S10": "SCORPIO",
    "SCORPIO S11": "SCORPIO",
    "SCORPIO S5": "SCORPIO",
    "SCORPIO S6": "SCORPIO",
    "SCORPIO S7": "SCORPIO",
    "SCORPIO S9": "SCORPIO",
}

# Function to normalize text
def normalize(text):
    # Handle NaN/None values or any non-string types
    if pd.isna(text) or text is None:
        return ""
    # Convert to string if it's not already a string
    if not isinstance(text, str):
        text = str(text)
    # Then proceed with normalization
    return re.sub(r'[^A-Z0-9 ]', '', text.upper())

# Function to extract Mahindra model
def extract_mahindra_model(text):
    txt = normalize(text)
    # First, correct aliases
    for bad, good in mahindra_alias_map.items():
        txt = txt.replace(bad, good)
    # Then search for any keyword
    for kw in sorted(mahindra_model_keywords, key=len, reverse=True):
        if kw in txt:
            return kw
    return "NOT FOUND"

# Apply extraction to the entire dataset
mahindra_data['mahindra_model'] = mahindra_data['rc_maker_model'].apply(extract_mahindra_model)

# Display extraction results
model_freq = mahindra_data['mahindra_model'].value_counts()
print("\nModel extraction results:")
print(model_freq.head(20))
print(f"\nModels found: {(mahindra_data['mahindra_model'] != 'NOT FOUND').sum()} out of {len(mahindra_data)} ({(mahindra_data['mahindra_model'] != 'NOT FOUND').sum()/len(mahindra_data)*100:.1f}%)")

# Create a final_model column using fallback logic
mahindra_data['final_model'] = mahindra_data['mahindra_model'].apply(lambda x: x if x != "NOT FOUND" else "NOT FOUND")

# Save results to CSV for review
output_path = '/home/bipin/Documents/kotak/car_model/clean_model/mahindra_models_extracted.csv'
mahindra_data.to_csv(output_path, index=False)
print(f"\nSaved results to {output_path}")

# Create a frequency mapping of original values to extracted models
mapping_df = mahindra_data.groupby('rc_maker_model')['mahindra_model'].agg(
    lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else "NOT FOUND"
).reset_index()
mapping_df.columns = ['original_value', 'mapped_model']
mapping_df['count'] = mapping_df['original_value'].map(mahindra_data['rc_maker_model'].value_counts())
mapping_df = mapping_df.sort_values('count', ascending=False)

# Save the mapping for review
mapping_path = '/home/bipin/Documents/kotak/car_model/clean_model/mahindra_model_mapping.csv'
mapping_df.to_csv(mapping_path, index=False)
print(f"Saved mapping to {mapping_path}")

# Save the keywords and alias map for reuse
with open('/home/bipin/Documents/kotak/car_model/clean_model/mahindra_keywords.py', 'w') as f:
    f.write("mahindra_model_keywords = [\n")
    for kw in sorted(mahindra_model_keywords):
        f.write(f"    \"{kw}\",\n")
    f.write("]\n\n")
    
    f.write("mahindra_alias_map = {\n")
    for bad, good in sorted(mahindra_alias_map.items()):
        f.write(f"    \"{bad}\": \"{good}\",\n")
    f.write("}\n")

print(f"\nKeywords and alias map saved to mahindra_keywords.py")